In [18]:
import simpy
import random
import numpy as np

NUM_PASSENGERS = 174

NUMBER_OF_INFECTED=0

class Passenger:
    def __init__(self, env, pid, infected=False):
        self.env = env
        self.pid = pid
        self.infected = infected
        self.incubating = False
        self.contagious = False
        self.newly_infected = False
        self.seat = None
        probabilities = [0.3, 0.5, 0.2]
        outcomes = [0, 1, 2]
        self.connections_left=random.choices(outcomes, probabilities)[0]

        if self.infected:
            self.env.process(self.start_incubation())

    def start_incubation(self):
        self.incubating = True
        incubation_time = random.uniform(180, 360)  # 3–6 hours in minutes
        yield self.env.timeout(incubation_time)
        self.incubating = False
        self.contagious = True

class Flight:
    def __init__(self, env, flight_time, flight_id, occupancy=NUM_PASSENGERS, infected_passengers=[]):
        self.env = env
        self.flight_time = flight_time
        self.flight_id = flight_id
        self.occupancy = occupancy
        self.infected_passengers = infected_passengers

        self.rows = (NUM_PASSENGERS + 5) // 6  # 6 seats per row (A–F)
        self.seating = {}  # seat -> Passenger
        self.bathroom = simpy.Resource(env, capacity=1)
        self.bathroom_infected = False
        self.bathroom_clean_time = 0

        self.passengers = []

        self._assign_seats()
        self.env.process(self.run_flight())

    def _assign_seats(self):
        # Create passengers
        passenger_pool = [Passenger(self.env, pid=i) for i in range(self.occupancy)]

        # Assign infected status
        infected_indices = random.sample(range(self.occupancy), len(self.infected_passengers))
        for idx in infected_indices:
            passenger_pool[idx].infected = True
            self.env.process(passenger_pool[idx].start_incubation())

        # Random seat assignment
        seats = [(r, c) for r in range(self.rows) for c in range(6)]
        assigned_seats = random.sample(seats, self.occupancy)
        for passenger, seat in zip(passenger_pool, assigned_seats):
            passenger.seat = seat
            self.seating[seat] = passenger
            self.passengers.append(passenger)

    def run_flight(self):
        # Start bathroom use
        for p in self.passengers:
            self.env.process(self.bathroom_behavior(p))

        # Handle row infections every 5 minutes
        while self.env.now < self.flight_time:
            yield self.env.timeout(5)
            self.handle_row_infection()

        # End of flight
        self.finish_flight()

    def handle_row_infection(self):
        for r in range(self.rows):
            row_passengers = [p for p in self.passengers if p.seat[0] == r]
            contagious_in_row = any(p.contagious for p in row_passengers)
            if contagious_in_row:
                for p in row_passengers:
                    if not (p.infected or p.incubating or p.newly_infected):
                        if random.random() < random.uniform(0.05, 0.15):  # Uniform chance
                            p.newly_infected = True
                            p.infected = True
                            self.env.process(p.start_incubation())

    def bathroom_behavior(self, passenger):
        while self.env.now < self.flight_time:
            lam = 2 if passenger.contagious else 0.5
            wait_time = random.expovariate(lam)
            yield self.env.timeout(wait_time)

            with self.bathroom.request() as req:
                yield req
                yield self.env.timeout(2)  # Duration in bathroom

                # Contaminate bathroom if contagious
                if passenger.contagious:
                    self.bathroom_infected = True
                    decay = random.expovariate(1/30)  # avg 30 mins infectious
                    self.bathroom_clean_time = self.env.now + decay

                # Infection from contaminated bathroom
                if self.bathroom_infected and self.env.now < self.bathroom_clean_time:
                    if not (passenger.infected or passenger.incubating or passenger.newly_infected):
                        if random.random() < 0.1:  # Uniform probability
                            passenger.newly_infected = True
                            passenger.infected = True
                            self.env.process(passenger.start_incubation())

    def finish_flight(self):
        infected = [p for p in self.passengers if p.infected]
        new_infected = len([p for p in self.passengers if p.newly_infected])

        print(f"\nFlight {self.flight_id} Summary at {self.env.now:.1f} min")
        print(f"Total passengers: {len(self.passengers)}")
        print(f"New infections: {new_infected}")


        return infected, new_infected

def run():
    env = simpy.Environment()
    # For test: infect 5 passengers
    infected_passenger_objs = [Passenger(env, pid=i, infected=True) for i in range(5)]
    f = Flight(env, flight_time=240, flight_id=1, occupancy=175, infected_passengers=infected_passenger_objs)
    # f.run_flight()
    env.run(until=300)


run()


ValueError: Sample larger than population or is negative

In [ ]:
import simpy
import random
import numpy as np
 
NUM_PASSENGERS = 175
 
class Passenger:
    def __init__(self, env, pid, infected=False):
        self.env = env
        self.pid = pid
        self.infected = infected
        self.incubating = False
        self.contagious = False
        self.newly_infected = False
        self.seat = None
        probabilities = [0.3, 0.5, 0.2]
        outcomes = [0, 1, 2]
        self.connections_left=random.choices(outcomes, probabilities)[0]

 
        if self.infected:
            self.env.process(self.start_incubation())
 
    def start_incubation(self):
        self.incubating = True
        incubation_time = random.uniform(180, 360)  # 3–6 hours in minutes
        yield self.env.timeout(incubation_time)
        self.incubating = False
        self.contagious = True
 
class Flight:
    def __init__(self, env, flight_time, flight_id, occupancy=NUM_PASSENGERS, infected_passengers=[]):
        self.env = env
        self.flight_time = flight_time
        self.flight_id = flight_id
        self.occupancy = occupancy
        self.infected_passengers = []
        self.seed_infected_passengers=infected_passengers
 
        self.rows = (NUM_PASSENGERS + 5) // 6  # 6 seats per row (A–F)
        self.seating = {}  # seat -> Passenger
        self.bathroom = simpy.Resource(env, capacity=1)
        self.bathroom_infected = False
        self.bathroom_clean_time = 0
        self.new_infected=0
        self.passengers = []
 
        self._assign_seats()
        self.env.process(self.run_flight())
 
    def _assign_seats(self):
        for p in self.seed_infected_passengers:
            self.infected_passengers.append(p)
            print(len(self.infected_passengers))
        # Create passengers
        passenger_pool = [Passenger(self.env, pid=i) for i in range(self.occupancy)]
 
        # Assign infected status
        infected_indices = random.sample(range(self.occupancy), len(self.infected_passengers))
        for idx in infected_indices:
            passenger_pool[idx].infected = True
            self.env.process(passenger_pool[idx].start_incubation())
 
        # Random seat assignment
        seats = [(r, c) for r in range(self.rows) for c in range(6)]
        assigned_seats = random.sample(seats, self.occupancy)
        for passenger, seat in zip(passenger_pool, assigned_seats):
            passenger.seat = seat
            self.seating[seat] = passenger
            self.passengers.append(passenger)
 
    def run_flight(self):
        # Start bathroom use
        for p in self.passengers:
            self.env.process(self.bathroom_behavior(p))
 
        # Handle row infections every 5 minutes
        while self.env.now < self.flight_time:
            yield self.env.timeout(5)
            self.handle_row_infection()
 
        # End of flight
        self.finish_flight()
 
    def handle_row_infection(self):
        for r in range(self.rows):
            row_passengers = [p for p in self.passengers if p.seat[0] == r]
            contagious_in_row = any(p.contagious for p in row_passengers)
            if contagious_in_row:
                for p in row_passengers:
                    if not (p.infected or p.incubating or p.newly_infected):
                        if random.random() < random.uniform(0.05, 0.15):  # Uniform chance
                            p.newly_infected = True
                            p.infected = True
                            self.env.process(p.start_incubation())
                            self.infected_passengers.append(p)
                            print(len(self.infected_passengers))
                            self.new_infected=self.new_infected+1
 
    def bathroom_behavior(self, passenger):
        while self.env.now < self.flight_time:
            lam = 2 if passenger.contagious else 0.5
            wait_time = random.expovariate(lam)
            yield self.env.timeout(wait_time)
 
            with self.bathroom.request() as req:
                yield req
                yield self.env.timeout(2)  # Duration in bathroom
 
                # Contaminate bathroom if contagious
                if passenger.contagious:
                    self.bathroom_infected = True
                    decay = random.expovariate(1/30)  # avg 30 mins infectious
                    self.bathroom_clean_time = self.env.now + decay
 
                # Infection from contaminated bathroom
                if self.bathroom_infected and self.env.now < self.bathroom_clean_time:
                    if not (passenger.infected or passenger.incubating or passenger.newly_infected):
                        if random.random() < 0.1:  # Uniform probability
                            passenger.newly_infected = True
                            passenger.infected = True
                            self.env.process(passenger.start_incubation())
 
    def finish_flight(self):
        infected = [p for p in self.passengers if p.infected]
        new_infected = len([p for p in self.passengers if p.newly_infected])
        departing = [p for p in self.passengers if not p.contagious]  # Placeholder logic
        continuing = []  # Placeholder: fill based on routing logic
 
        print(f"\nFlight {self.flight_id} Summary at {self.env.now:.1f} min")
        print(f"Total passengers: {len(self.passengers)}")
        print(f"New infections: {new_infected}")
        print(f"Departing passengers: {len(departing)}")
        print(f"Placeholder - continuing passengers: {len(continuing)}")
 
        return infected,new_infected
 
def run():
    env = simpy.Environment()
    # For test: infect 5 passengers
    infected_passenger_objs = [Passenger(env, pid=i, infected=True) for i in range(5)]
    f = Flight(env, flight_time=240, flight_id=1, occupancy=175, infected_passengers=infected_passenger_objs)
    env.run(until=300)
 
run()

1
2
3
4
5

5

6

Flight 1 Summary at 240.0 min
Total passengers: 175
New infections: 1
Departing passengers: 174
Placeholder - continuing passengers: 0


In [ ]:
def run():
    SIM_DURATION = 48 * 60  # 48 hours in minutes
    FLIGHT_DURATION = 4 * 60  # 4-hour flights
    OCCUPANCY = 175

    env = simpy.Environment()
    flight_id = 1
    flights = []

    # Initial infection: seed 5 infected passengers
    initial_infected = [Passenger(env, pid=i, infected=True) for i in range(5)]
    total_infected = len(initial_infected)
    # Create first flight
    first_flight = Flight(env, departure= 0, arrival flight_time=FLIGHT_DURATION, flight_id=flight_id,
                          occupancy=OCCUPANCY, infected_passengers=initial_infected)
    flights.append(first_flight)

    def generate_layover():
        return max(30, min(180, random.normalvariate(105, 25)))

layover = generate_layover()
departure_time = env.now + layover

env.process(schedule_next_flight(departure_time, infected_passengers))
    # Flight launcher coroutine
    def flight_chain(flight):
        yield env.timeout(FLIGHT_DURATION)
        infected, new_infected = flight.finish_flight()
        total_infected+=total_infected

        if infected and env.now + FLIGHT_DURATION < SIM_DURATION:
            nonlocal flight_id
            flight_id += 1
            next_flight = Flight(env, flight_time=FLIGHT_DURATION, flight_id=flight_id,
                                 occupancy=OCCUPANCY, infected_passengers=infected)
            flights.append(next_flight)
            env.process(flight_chain(next_flight))

    # Start simulation
    env.process(flight_chain(first_flight))
    env.run(until=SIM_DURATION)

    # Summary
    print("\n==== Simulation Complete ====")
    print(f"Total Flights Simulated: {len(flights)}")
    total_infected = sum(len(f.infected_passengers) for f in flights)
    print(f"Total Infected (cumulative, includes duplicates): {total_infected}")



Flight 1 Summary at 240.0 min
Total passengers: 175
New infections: 3
Departing passengers: 173
Placeholder - continuing passengers: 0
